In [1]:
import pandas as pd
import gzip
import os
import matplotlib.pyplot as plt

GTEX_path = './GTEX/'
Xenium_path = './XeniumPanels/'

In [2]:
GTEX_dir = os.listdir(GTEX_path)
GTEX_files = [file for file in GTEX_dir if file.endswith('.gct.gz')]
Xenium_dir = os.listdir(Xenium_path)
Xenium_files = [file for file in Xenium_dir if file.endswith('.csv')]

In [3]:
print(Xenium_files)
Xenium_titles = Xenium_files.copy()
for i in range(len(Xenium_titles)):
    Xenium_titles[i] = Xenium_titles[i].split('_')[1]
print(Xenium_titles)

['Xenium_hSkin_preview_metadata.csv', 'Xenium_hBrain_v1_metadata.csv', 'Xenium_hColon_preview_metadata.csv', 'Xenium_hBreast_v1_metadata.csv', 'Xenium_hLung_v1_metadata.csv', 'Xenium_hMulti_v1_metadata.csv']
['hSkin', 'hBrain', 'hColon', 'hBreast', 'hLung', 'hMulti']


In [12]:
for x in range(len(Xenium_files)):
    tissue_series_list = []
    print(Xenium_titles[x])
    
    gene_list_path = Xenium_path+Xenium_files[x]
    df_genes = pd.read_csv(gene_list_path)
            
    for g in GTEX_files:

        # open and read the gzipped file
        with gzip.open(GTEX_path+g, 'rt') as file:
            # skip the first 2 rows (header information) and read the data into a DataFrame
            df = pd.read_csv(file, sep='\t', skiprows=2, index_col=0)

        ensemble_ids = df_genes['Ensembl_ID'].unique()
        df['Name'] = df['Name'].str.split('.').str[0]
        filtered_df = df[df['Name'].isin(ensemble_ids)]

        filtered_df.set_index('Name', inplace=True)

        columns_to_average = filtered_df.columns.difference(['Description'])
        tissue_series_list.append(filtered_df[columns_to_average].mean(axis=1))
    
    # df_plot = pd.concat(tissue_series_list, axis=1)
    # df_plot.columns = GTEX_files
    # print(df_plot)
    GTEX_names = GTEX_files.copy()
    for i in range(len(GTEX_names)):
        GTEX_names[i] = '_'.join(GTEX_names[i].split('_')[4:]).replace('.gct.gz', '')
    # print(GTEX_names)

    # plt.figure(figsize=(20, 6))
    # box = df_plot.boxplot(flierprops=dict(markersize=3), grid=False)
    # box.set_xticklabels(GTEX_names, rotation=45, ha='right')
    # plt.ylabel('Log Scaled Expression Level')
    # plt.yscale('log')
    # plt.title('GTEX-Xenium ' + Xenium_titles[x] + ' Panel Box Plot')
    # plt.axhline(y=10, color='r', linestyle='--')
    # plt.axhline(y=1, color='b', linestyle=':') 
    # plt.show()

hSkin
260
hBrain
265
hColon
322
hBreast
281
hLung
289
hMulti
378


In [31]:
df_csv = df_genes[['Ensembl_ID', 'Gene']]
df_csv.set_index('Ensembl_ID', inplace=True)
for name in GTEX_names:
    df_csv[name] = None

print(df_csv)

print(df_csv.at['ENSG00000121270', 'thyroid'])

                   Gene thyroid brain_caudate_basal_ganglia brain_amygdala  \
Ensembl_ID                                                                   
ENSG00000121270  ABCC11    None                        None           None   
ENSG00000130234    ACE2    None                        None           None   
ENSG00000213088   ACKR1    None                        None           None   
ENSG00000107796   ACTA2    None                        None           None   
ENSG00000163017   ACTG2    None                        None           None   
...                 ...     ...                         ...            ...   
ENSG00000243566   UPK3B    None                        None           None   
ENSG00000038427    VCAN    None                        None           None   
ENSG00000155659   VSIG4    None                        None           None   
ENSG00000110002   VWA5A    None                        None           None   
ENSG00000110799     VWF    None                        None     

/tmp/ipykernel_2804650/1149195582.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_csv[name] = None
/tmp/ipykernel_2804650/1149195582.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_csv[name] = None
/tmp/ipykernel_2804650/1149195582.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ret

In [34]:
print(tissue_series_list[0])

Name
ENSG00000188290    26.399315
ENSG00000049249     0.164836
ENSG00000162493     9.542253
ENSG00000142973    36.674208
ENSG00000162595     5.088309
                     ...    
ENSG00000155659    21.954740
ENSG00000169083     4.727444
ENSG00000147257    46.542922
ENSG00000198223     0.000000
ENSG00000185291     0.000000
Length: 378, dtype: float64
